In [399]:
### Calculate (future) ACE from forecasts of a single intensity model using cyclone a-tracks
### Uses linear interpolation to get values of 6h intervals

# download latest a-tracks manually for basin of interest (no automation yet)
# and change names of atrack files below

# note: make sure to gunzip them
# https://ftp.nhc.noaa.gov/atcf/aid_public/?C=M;O=D
    
### DOWNLOAD LATEST ATRACKS AND UPDATE THE FOLLOWING TWO VARIABLES EVERY RUN OF THE NOTEBOOK
atrack_files = [
    'aal152023.dat'
]

# latest real time ACE (for North Atlantic)
# from http://tropical.atmos.colostate.edu/Realtime/
real_time_ACE = 110.8

In [400]:
##### CONFIG BASIN AND INTENSITY MODEL
match_basin = 'AL'
# consensus model
match_model = 'IVCN'

In [401]:
ab_column_names = [
    'BASIN',
    'CY',
    'YYYYMMDDHH',
    'TECHNUM/MIN',
    'TECH',
    'TAU',
    'LatN/S',
    'LonE/W',
    'VMAX',
    'MSLP',
    'TY',
    'RAD',
    'WINDCODE',
    'RAD1',
    'RAD2',
    'RAD3',
    'RAD4',
    'POUTER',
    'ROUTER',
    'RMW',
    'GUSTS',
    'EYE',
    'SUBREGION',
    'MAXSEAS',
    'INITIALS',
    'DIR',
    'SPEED',
    'STORMNAME',
    'DEPTH',
    'SEAS',
    'SEASCODE',
    'SEAS1',
    'SEAS2',
    'SEAS3',
    'SEAS4',
    'USERDEFINED1',
    'userdata1',
    'USERDEFINED2',
    'userdata2',
    'USERDEFINED3',
    'userdata3',
    'USERDEFINED4',
    'userdata4',
    'USERDEFINED5',
    'userdata5'
]

In [402]:
import pandas as pd
from datetime import datetime

track_line_lists = []

# read the tracks
for atrack_file in atrack_files:
    with open(atrack_file, 'r') as file:
        lines = file.readlines()
    for line in lines:
        line_list = line.split(',')
        line_list_trim = [x.strip() for x in line_list]
        line_list_trim_rempty = line_list_trim
        
        # trim empty columns on right
        # a tracks have an extra comma before EOL for some reason (should have max 45 columns)
        line_list_trim_rempty.reverse()
        line_list_trim_empty = []
        for [i, v] in enumerate(line_list_trim_rempty):
            if v:
                line_list_trim_empty = line_list_trim_rempty[i:]
                break
        line_list_trim_empty.reverse()
        # convert init time strings to timestamp format
        line_list_trim_empty[2] = datetime.strptime(line_list_trim_empty[2], '%Y%m%d%H')
            
        track_line_lists.append(line_list_trim_empty)

In [403]:
# convert the line lists to a dataframe
df = pd.DataFrame(track_line_lists, columns=ab_column_names)

In [404]:
# get the last matching forecasts
df_match = df.loc[(df['BASIN'] == match_basin) & (df['TECH'] == match_model)]

newest_matching_cyclone_forecast_dfs = []
for cyclone_num in df_match['CY'].unique():
    df_cyclone = df_match.loc[(df['CY'] == cyclone_num)]
    newest_init_time = df_cyclone['YYYYMMDDHH'].max()
    newest_forecast = df_cyclone.loc[(df['YYYYMMDDHH'] == newest_init_time)]
    # make sure these columns are numeric
    # convert to numeric
    col_numeric_names = ["CY", "TAU", "VMAX", "MSLP"]
    for col_numeric_name in col_numeric_names:
        newest_forecast = newest_forecast.astype({col_numeric_name:'int'})
    # append to list of cyclone forecast dfs
    newest_matching_cyclone_forecast_dfs.append(newest_forecast)

In [405]:
import datetime as datetime
# interpolate to provide 06 taus
interp_intensity_cyclone_dfs = []

last_valid_time = None

for cyclone_df in newest_matching_cyclone_forecast_dfs:
    forecast_taus = cyclone_df['TAU'].unique()
    min_tau = forecast_taus.min()
    max_tau = forecast_taus.max()
    taus_6h = list(range(min_tau, max_tau + 6, 6))
    taus_missing = []
    for tau in taus_6h:
        if tau not in forecast_taus:
            taus_missing.append(tau)
    for missing_tau in taus_missing:
        # get prev tau
        prev_tau = missing_tau
        while prev_tau >= min_tau:
            prev_tau = prev_tau - 1
            if prev_tau in forecast_taus:
                break
        next_tau = missing_tau
        while next_tau <= max_tau:
            next_tau = next_tau + 1
            if next_tau in forecast_taus:
                break
        prev_intensity = cyclone_df.loc[(cyclone_df['TAU'] == prev_tau)]['VMAX'].values[0]
        next_intensity = cyclone_df.loc[(cyclone_df['TAU'] == next_tau)]['VMAX'].values[0]
        interpolated_intensity = int(round((prev_intensity + next_intensity) / 2.0))
        cyclone_df = cyclone_df.reset_index(drop=True)
        prev_idx = cyclone_df.loc[(cyclone_df['TAU'] == prev_tau)].index.values[0]
        # copy row
        interp_row = cyclone_df.iloc[prev_idx].copy()
        # only manipulate the TAU and INTENSITY so we don't have to copy the rest of the values
        # the long/lat and rest of data have not been interpolated....
        interp_row['TAU'] = missing_tau
        interp_row['VMAX'] = interpolated_intensity
        # add row
        #cyclone_df = cyclone_df.append(interp_row, ignore_index=True)
        cyclone_df = pd.concat([cyclone_df, pd.DataFrame([interp_row])], ignore_index=True)
    cyclone_df = cyclone_df.reset_index(drop=True)
    cyclone_df.sort_values(by=['TAU'], inplace=True)
    cyclone_df = cyclone_df.reset_index(drop=True)
    interp_intensity_cyclone_dfs.append(cyclone_df)
    # cyclone number
    cyclone_num = cyclone_df['CY'].iloc[-1]
    basin = cyclone_df.iloc[-1]['BASIN']
    init_time = cyclone_df.iloc[0]['YYYYMMDDHH']
    valid_time = init_time_str + datetime.timedelta(hours=int(max_tau))
    # print the forecast times
    print(f"{basin}{cyclone_num}: 0-{max_tau}H : {init_time} to {valid_time}")
    if last_valid_time == None:
        last_valid_time = valid_time
    if valid_time > last_valid_time:
        last_valid_time = valid_time

print("")
print("Longest forecast covers up until:", last_valid_time)

AL15: 0-144H : 2023-09-19 18:00:00 to 2023-09-25 00:00:00

Longest forecast covers up until: 2023-09-25 00:00:00


In [406]:
# exclude storms less than 35 kt
# exclude forecasts at 00h (since this may already be included in real time ACE)
ACE_min_kt = 35
storm_intensities_above_min = {}
for cyclone_df in interp_intensity_cyclone_dfs:
    cyclone_num = cyclone_df['CY'].iloc[-1]
    basin = cyclone_df.iloc[-1]['BASIN']
    storm_name_str = f'{basin}{cyclone_num}'
    min_kt_rows = cyclone_df.loc[(cyclone_df['VMAX'] >= ACE_min_kt) & (cyclone_df['TAU'] > 0)]['VMAX']
    storm_intensities_above_min[storm_name_str] = list(min_kt_rows.values)

In [407]:
import numpy as np
forecasts_ace = {}
for [storm_name, vmax] in storm_intensities_above_min.items():
    forecast_storm_ace = (pow(10, -4) * np.sum(np.power(vmax, 2)))
    forecasts_ace[storm_name] = round(forecast_storm_ace,2)

In [408]:
forecast_total_ace = 0
print(f"Future contributions (up until {last_valid_time}) of cyclone ACE values in {match_basin} basin (including INVESTs)\n",
      f"   from latest forecast using {match_model} (excluding 00h) by interpolating intensities")
for [storm_name, ace] in forecasts_ace.items():
    print(f"  ACE(future) for {storm_name} = {ace}")
    forecast_total_ace += ace
print(f"  Total ACE(future): {forecast_total_ace:4.2f}")
total_ace_until_forecast = real_time_ACE + forecast_total_ace
print("")
print("Real-time ACE:", real_time_ACE)
print(f"Total ACE (real-time + forecast) up until {last_valid_time}: {total_ace_until_forecast}")

Future contributions (up until 2023-09-25 00:00:00) of cyclone ACE values in AL basin (including INVESTs)
    from latest forecast using IVCN (excluding 00h) by interpolating intensities
  ACE(future) for AL15 = 9.64
  Total ACE(future): 9.64

Real-time ACE: 110.8
Total ACE (real-time + forecast) up until 2023-09-25 00:00:00: 120.44


In [409]:
print(storm_intensities_above_min)

{'AL15': [86, 87, 86, 84, 80, 77, 74, 71, 65, 59, 52, 45, 46, 48, 49, 50, 53, 56, 55, 54, 52, 50, 51, 52]}


In [410]:
print(interp_intensity_cyclone_dfs)

[   BASIN  CY          YYYYMMDDHH TECHNUM/MIN  TECH  TAU LatN/S LonE/W  VMAX  \
0     AL  15 2023-09-19 18:00:00          03  IVCN    0     0N     0W    85   
1     AL  15 2023-09-19 18:00:00          03  IVCN    6     0N     0W    86   
2     AL  15 2023-09-19 18:00:00          03  IVCN   12     0N     0W    87   
3     AL  15 2023-09-19 18:00:00          03  IVCN   18     0N     0W    86   
4     AL  15 2023-09-19 18:00:00          03  IVCN   24     0N     0W    84   
5     AL  15 2023-09-19 18:00:00          03  IVCN   30     0N     0W    80   
6     AL  15 2023-09-19 18:00:00          03  IVCN   36     0N     0W    77   
7     AL  15 2023-09-19 18:00:00          03  IVCN   42     0N     0W    74   
8     AL  15 2023-09-19 18:00:00          03  IVCN   48     0N     0W    71   
9     AL  15 2023-09-19 18:00:00          03  IVCN   54     0N     0W    65   
10    AL  15 2023-09-19 18:00:00          03  IVCN   60     0N     0W    59   
11    AL  15 2023-09-19 18:00:00          03  IVCN 

In [411]:

import pandas as pd
import numpy as np
from datetime import datetime

# Define a function to convert the components into datetime objects
def convert_to_datetime(components, debug = False):
    # only use first time
    try:
        c = "".join(components.astype(str))
        year = int(c[0:4])
        month = int(c[5:7])
        day = int(c[8:10])
        hour = int(c[11:13])
        minute = int(c[14:16])
        second = int(c[17:])
        dt = datetime(year, month, day, hour, minute, second)
        return dt
    except:
        if debug:
            print("err")
            print(components)
        return None
    
def extract_first_num(components):
    try:
        return components[0].astype(int)
    except:
        return None

def extract_first_str(components):
    try:
        return "".join(components[0].astype(str))
    except:
        return None

def extract_first_byte(components):
    try:
        return "".join(components[0].astype(str))
    except:
        return None

In [412]:
# Calculate historical ACE energies from best tracks and use this to calculate expected ACE values from the above values

# from https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/
best_tracks_csv = 'ibtracs.NA.list.v04r00.csv'

# Read the CSV file into a DataFrame without automatically assigning column names
df = pd.read_csv(best_tracks_csv, header=0, skiprows=[1], low_memory=False)


In [413]:
# Create a dictionary that specifies the data types for each column
dtype_mapping = {
    'LAT': float,
    'LON': float,
    'WMO_WIND': float,
    'WMO_PRES': float,
    'DIST2LAND': float,
    'LANDFALL': float,
    'STORM_SPEED': float,
    'STORM_DIR': float
}

# Apply dtype conversion to specific columns, treating non-numeric values as NaN
for column, data_type in dtype_mapping.items():
    df[column] = pd.to_numeric(df[column], errors='coerce').astype(data_type)

# convert time to datetime
df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'])


In [414]:

# Create a mask to filter the rows for only 6 hour times
mask = (df['ISO_TIME'].dt.hour.isin([0, 6, 12, 18])) & (df['ISO_TIME'].dt.minute == 0) & (df['ISO_TIME'].dt.second == 0)

# Apply the mask to filter the DataFrame
filtered_df = df[mask].copy()

# drop all but relevant column's for speed
selected_columns = ['SEASON', 'ISO_TIME', 'WMO_WIND']
filtered_df = filtered_df[selected_columns]

In [415]:
df_min_35kt = filtered_df.loc[(filtered_df['WMO_WIND'] >= 35)]
df_min_35kt.reset_index()

,index,SEASON,ISO_TIME,WMO_WIND
0,12,1851,1851-06-25 00:00:00,80.0
1,14,1851,1851-06-25 06:00:00,80.0
2,16,1851,1851-06-25 12:00:00,80.0
3,18,1851,1851-06-25 18:00:00,80.0
4,20,1851,1851-06-26 00:00:00,70.0
...,...,...,...,...
40176,125434,2022,2022-11-10 00:00:00,65.0
40177,125436,2022,2022-11-10 06:00:00,65.0
40178,125439,2022,2022-11-10 12:00:00,55.0
40179,125441,2022,2022-11-10 18:00:00,40.0


In [416]:
#### if desired, use a manually specified time (must comment out the below line using last_valid_time)
ace_time_end_str = "2023-09-24 06:00:00"
ace_time = datetime.fromisoformat(ace_time_end_str)

# otherwise, use the last valid_time
ace_time = last_valid_time
####


# Calculate ACE for up to ace_time_end_str for all years

ace_time_month = ace_time.month
ace_time_day = ace_time.day

season_ace = {}
season_kt = {}
kt_at_time = {}
season_ace_to_time_end = {}
season_ace_above_real_time = {}

for idx in range(0, len(df_min_35kt)):
    r = df_min_35kt.iloc[idx]
    t = r['ISO_TIME']
    w = r['WMO_WIND']
    kt_at_time[t] = w

years = list(set(list(df_min_35kt['SEASON'].values)))
    
for season in years:
    df_season_min_35kt = df_min_35kt.loc[(df_min_35kt['SEASON'] == season)]
    kts = list(df_season_min_35kt['WMO_WIND'].values)
    season_kt[season] = kts
    ace = (pow(10, -4) * np.sum(np.power(kts, 2)))
    season_ace[season] = ace
    
    # exclude ace that are not representative for population statistics when they are below real time ACE
    if ace >= real_time_ACE:
        season_ace_above_real_time[season] = ace
    
    # calculate ACE until ace_time_end_str for each year
    df_season_to_time_end_min_35kt = df_season_min_35kt.loc[
        (df_season_min_35kt['ISO_TIME'].dt.month < ace_time_month) | 
        (
            (df_season_min_35kt['ISO_TIME'].dt.month == ace_time_month) &
            (df_season_min_35kt['ISO_TIME'].dt.day <= ace_time_day)
        )
    ]
    kts = list(df_season_to_time_end_min_35kt['WMO_WIND'].values)
    ace = (pow(10, -4) * np.sum(np.power(kts, 2)))
    season_ace_to_time_end[season] = ace

print(f"Done calculating ACE for each year up until {last_valid_time.month}/{last_valid_time.day}")

Done calculating ACE for each year up until 9/25


In [417]:
# use this to normalize a 1-d array
def normalize(matrix):
    norm = np.linalg.norm(matrix, 1)
    matrix = matrix/norm 
    return matrix

# use exponential decay (the difference passed should always be positive)
def custom_kernel(difference, rate):
    #return (1.0 / difference)
    return np.exp(-rate * difference)

def get_min_pair_by_value(d):
    # Find the key/value pair with the minimum value from a dict
    return min(d.items(), key=lambda x: x[1])

def mean_squared_error(predictions, actual_values):
    # Ensure both dictionaries have the same keys
    common_keys = set(predictions.keys()) & set(actual_values.keys())
    
    # Calculate the squared differences and sum them
    squared_diff_sum = sum((predictions[key] - actual_values[key])**2 for key in common_keys)
    
    # Calculate the mean squared error
    mse = squared_diff_sum / len(common_keys)
    
    return mse

In [418]:
### This should be done regularly (after every day?) to get lower variance and proper decay values considering the time until to the end of the season

### TUNING & VALIDATION for decay rates and expected ACE (for this far out in year) ###

### do tuning on a partial dataset to calculate MSE and then do validation on the remaining dataset
### to make sure MSE is not too far off (this will be an estimate of the additional error/uncertainty in the tuning)

import random

years = list(season_ace_to_time_end.keys())
years_above_real_time_ace = sorted(list(season_ace_above_real_time.keys()))

# Calculate the size of the first list (80%)
tuning_list_size = int(len(years) * 0.8)

# Randomly shuffle the original list
random.shuffle(years)

# Split the list into two new lists
years_tuning = years[:tuning_list_size]
years_validation = years[tuning_list_size:]
print(f"Tuning years: ", years_tuning)
print(f"Validation years: ", years_validation)

Tuning years:  [1851, 2015, 1864, 1867, 1976, 2009, 1955, 1969, 1880, 1881, 1897, 1988, 1919, 2000, 2006, 1877, 1909, 2012, 1920, 1981, 1972, 2019, 1934, 1974, 1912, 1868, 1876, 1893, 1905, 1921, 1858, 1855, 1900, 1989, 1966, 1859, 2007, 1856, 1971, 1964, 1960, 2022, 1942, 1983, 1949, 1973, 2020, 1987, 1889, 2021, 1894, 1998, 1910, 1903, 1870, 1872, 1924, 1861, 1852, 1967, 1931, 1918, 1865, 1984, 1928, 1887, 1944, 2010, 1917, 1891, 1886, 1907, 1927, 1908, 1993, 1898, 1991, 2001, 1954, 1947, 1914, 1922, 1994, 2005, 1963, 1890, 1956, 1896, 1916, 1871, 1936, 1970, 1979, 1913, 1929, 1935, 1996, 2016, 1892, 1926, 1869, 1938, 2003, 1883, 1901, 1888, 1933, 1985, 2002, 1992, 1882, 1977, 1911, 1904, 1857, 1878, 1990, 1957, 1980, 1902, 1958, 1884, 1930, 1943, 2014, 1925, 1951, 1965, 1986, 1874, 1940, 2008, 2013, 2018, 1968, 1999, 1952]
Validation years:  [1982, 1879, 1899, 1860, 1978, 1995, 1885, 2004, 1961, 1862, 1866, 1941, 1962, 1873, 1950, 1932, 1895, 1946, 2017, 1939, 1915, 1997, 1923, 1863

In [419]:
# exponential decay weights
rates_to_try = list(np.round(np.arange(0.1, 0.2, 0.001),3))
#rates_to_try = [0.0001, 0.001, 0.01, 0.1, 1]
mse_by_rate = {}

# Calculate weights for each decay rate
for rate in rates_to_try:
    expected_ACE_by_tuning_year = {}
    # tune over only years for validation set
    for tuning_year in years_tuning:
        # for tuning this is the season_ace_to_time_end[tuning_year]
        tuning_ACE = season_ace_to_time_end[tuning_year]

        tuning_diff_season_ace_to_time_end = {}

        for [year, ace] in season_ace_to_time_end.items():
            if year in years_tuning and year != tuning_year:
                tuning_diff_season_ace_to_time_end[year] = tuning_ACE - ace

        # Calculate weights for each year
        weights_normalized = []
        weights = []
        for [year, diff] in tuning_diff_season_ace_to_time_end.items():        
            # Calculate weights using the custom kernel function
            weight = custom_kernel(np.abs(diff), rate)
            weights.append(weight)

        # Calculate the normalized weights
        weights_normalized = normalize(weights)

        # Calculate expected ACE using normalized weights
        expected_ACE = 0.0
        for [x, year] in list(enumerate(tuning_diff_season_ace_to_time_end)):
            expected_ACE += (weights_normalized[x] * season_ace[year])
        #print(tuning_diff_season_ace_to_time_end)

        expected_ACE_by_tuning_year[tuning_year] = expected_ACE
    
    # calculate the MSE for this rate
    #print(expected_ACE_by_tuning_year)
    mse = mean_squared_error(expected_ACE_by_tuning_year, season_ace)
    mse_by_rate[rate] = mse

In [420]:
tuning_min_pair = get_min_pair_by_value(mse_by_rate)
print(f"(Validation) Rate , MSE (minimum) for tuning dataset: {tuning_min_pair}")

(Validation) Rate , MSE (minimum) for tuning dataset: (0.136, 525.5818424841025)


In [421]:
# TUNING above has minimum MSE for:
# Rate: 0.14, MSE: 578.8630529802942

In [422]:
## VALIDATION
# exponential decay weights
rates_to_try = list(np.round(np.arange(0.05, 0.2, 0.001),3))
#rates_to_try = [0.0001, 0.001, 0.01, 0.1, 1]
mse_by_rate = {}

# Calculate weights for each decay rate
for rate in rates_to_try:
    expected_ACE_by_validation_year = {}
    for validation_year in years_validation:
        # for validation this is the season_ace_to_time_end[validation_year]
        validation_ACE = season_ace_to_time_end[validation_year]

        validation_diff_season_ace_to_time_end = {}

        for [year, ace] in season_ace_to_time_end.items():
            if year in years_validation and year != validation_year:
                validation_diff_season_ace_to_time_end[year] = validation_ACE - ace

        # Calculate weights for each year
        weights_normalized = []
        weights = []
        for [year, diff] in validation_diff_season_ace_to_time_end.items():        
            # Calculate weights using the custom kernel function
            weight = custom_kernel(np.abs(diff), rate)
            weights.append(weight)

        # Calculate the normalized weights
        weights_normalized = normalize(weights)

        # Calculate expected ACE using normalized weights
        expected_ACE = 0.0
        for [x, year] in list(enumerate(validation_diff_season_ace_to_time_end)):
            expected_ACE += (weights_normalized[x] * season_ace[year])
        #print(validation_diff_season_ace_to_time_end)

        expected_ACE_by_validation_year[validation_year] = expected_ACE
    
    # calculate the MSE for this rate
    #print(expected_ACE_by_validation_year)
    mse = mean_squared_error(expected_ACE_by_validation_year, season_ace)
    mse_by_rate[rate] = mse
    

In [423]:
validation_min_pair = get_min_pair_by_value(mse_by_rate)
print(f"(Validation) Rate : MSE (minimum) for tuning dataset: {validation_min_pair}")

(Validation) Rate : MSE (minimum) for tuning dataset: (0.091, 417.0722495923594)


In [424]:
# for validation, the minimum MSE:
# Rate: 0.102, MSE: 439.9012525009955
# from the validation-tuning set above, the tuning Rate: 0.14, MSE: 450.0207460685012

rmse_tuning = pow(tuning_min_pair[1], 0.5)
rmse_validation = pow(validation_min_pair[1], 0.5)
diff_rmse = abs(rmse_tuning - rmse_validation)
print("RMSE tuning:", rmse_tuning)
print("RMSE validation:", rmse_tuning)                 
print("Difference of RMSE(tuning) - RMSE(validation):", pow(diff_rmse,0.5))

RMSE tuning: 22.925571802773042
RMSE validation: 22.925571802773042
Difference of RMSE(tuning) - RMSE(validation): 1.5821583308412621


In [425]:
# FINAL TUNING

# exponential decay weights
rates_to_try = list(np.round(np.arange(0.05, 0.2, 0.001),3))
#rates_to_try = [0.0001, 0.001, 0.01, 0.1, 1]
mse_by_rate = {}

# Calculate weights for each decay rate
for rate in rates_to_try:
    expected_ACE_by_tuning_year = {}
    # tune over all years
    for tuning_year in years:
        # for tuning this is the season_ace_to_time_end[tuning_year]
        tuning_ACE = season_ace_to_time_end[tuning_year]

        tuning_diff_season_ace_to_time_end = {}

        for [year, ace] in season_ace_to_time_end.items():
            if year in years and year != tuning_year:
                tuning_diff_season_ace_to_time_end[year] = tuning_ACE - ace

        # Calculate weights for each year
        weights_normalized = []
        weights = []
        for [year, diff] in tuning_diff_season_ace_to_time_end.items():        
            # Calculate weights using the custom kernel function
            weight = custom_kernel(np.abs(diff), rate)
            weights.append(weight)

        # Calculate the normalized weights
        weights_normalized = normalize(weights)

        # Calculate expected ACE using normalized weights
        expected_ACE = 0.0
        for [x, year] in list(enumerate(tuning_diff_season_ace_to_time_end)):
            expected_ACE += (weights_normalized[x] * season_ace[year])
        #print(tuning_diff_season_ace_to_time_end)

        expected_ACE_by_tuning_year[tuning_year] = expected_ACE
    
    # calculate the MSE for this rate
    #print(expected_ACE_by_tuning_year)
    mse = mean_squared_error(expected_ACE_by_tuning_year, season_ace)
    mse_by_rate[rate] = mse
    #print(f"Rate: {rate}, MSE: {mse}")

final_tuning_min_pair = get_min_pair_by_value(mse_by_rate)
print(f"(Tuning Final) Rate : MSE (minimum) for all past years: {final_tuning_min_pair}")

(Tuning Final) Rate : MSE (minimum) for all past years: (0.171, 481.5783055586021)


In [426]:
# minimum is Rate: 0.147, MSE: 518.519969751831
mse_final_tuning = final_tuning_min_pair[1]
rate_final_tuning = final_tuning_min_pair[0]
print("Rate final tuning:", rate_final_tuning)
print("MSE final tuning:", mse_final_tuning)
rmse_final_tuning = pow(mse_tuning,0.5)
print("RMSE final tuning:", var_final_tuning)

Rate final tuning: 0.171
MSE final tuning: 481.5783055586021
RMSE final tuning: 21.777957351839856


In [429]:
from scipy.stats import norm
from collections import Counter

# Calculate the critical value for the specified confidence level
critical_value = norm.ppf((1 + confidence_level) / 2)

# real-time ACE plus forecast ace (until ace_time_end_str)
# use a manually entered value
present_ACE = 122.65
# or the value from the forecast
present_ACE = total_ace_until_forecast

def calculate_expected_ACE(this_season_end = season_ace_to_time_end, repeat_counts = None):
    diff_season_ace_to_time_end = {}

    for [year, ace] in this_season_end.items():
        diff_season_ace_to_time_end[year] = present_ACE - ace

    # exponential decay weights
    #rates_to_try = list(np.round(np.arange(0.080, 0.205, 0.005),3))
    rates_to_try = [rate_final_tuning]

    expected_ACE_by_rate = {}
    weights_normalized_by_rate = {}
    # Calculate weights for each decay rate
    for rate in rates_to_try:
        # Calculate weights for each year
        weights_normalized = []
        weights = []
        for [year, diff] in diff_season_ace_to_time_end.items():        
            # Calculate weights using the custom kernel function
            weight = custom_kernel(np.abs(diff), rate)
            weights.append(weight)

        # Calculate the normalized weights
        weights_normalized = normalize(weights)

        weights_normalized_by_rate[rate] = weights_normalized

        # Calculate expected ACE using normalized weights
        expected_ACE = 0.0
        for [x, year] in list(enumerate(diff_season_ace_to_time_end)):
            if repeat_counts:
                repeat_count = repeat_counts[year]
                expected_ACE += (repeat_count * (weights_normalized[x] * season_ace[year]))
            else:
                expected_ACE += (weights_normalized[x] * season_ace[year])

        expected_ACE_by_rate[rate] = expected_ACE

    expected_ACE = expected_ACE_by_rate[rate_final_tuning]

    return expected_ACE


expected_ACE = calculate_expected_ACE()
    
pop_ace_values = list(season_ace_above_real_time.values())
ace_stddev = np.std(pop_ace_values)
ace_mean =  np.mean(pop_ace_values)
z_score = (expected_ACE - ace_mean) / ace_stddev


sorted_years = sorted(years)

print(f"Forecasts until:", last_valid_time)
print(f"ACE, Real-time + storm model forecast: {total_ace_until_forecast}")
print("Tuned decay rate:", rate_final_tuning)
print("")
print(f"Expected ACE (weight historical ACE using exponentially decay): {expected_ACE:3.2f}")


# Specify the confidence level (e.g., 90% CI)
confidence_level = 0.7

# use bootstrapping instead for CI

# Number of bootstrap samples to generate
num_samples = 10000

# Calculate the statistic of interest (e.g., mean) for the original data
original_statistic = expected_ACE

# Initialize an array to store resampled statistics
resampled_statistics = np.empty(num_samples)

# Perform bootstrapping
for i in range(num_samples):
    # Generate a resampled dataset by sampling with replacement
    
    # cannot do random choice on a dict...
    #resampled_data = np.random.choice(season_ace_above_real_time, size=len(season_ace_above_real_time), replace=True)
    
    num_samples = len(season_ace_above_real_time)  # Number of samples you want to draw
    # Randomly select keys from the dictionary with replacement
    random_keys = random.choices(list(season_ace_to_time_end.keys()), k=num_samples)
    # Use the selected keys to access corresponding values
    random_values = [season_ace_to_time_end[key] for key in random_keys]
    
    # since I need to pass a dict that should have multiple keys repeated, do something extra to keep track of the 'repeats'
    repeat_counts = dict(Counter(random_keys))
    
    resampled_data = {key: value for key, value in zip(random_keys, random_values)}
    
    # Calculate the statistic of interest for the resampled dataset
    resampled_statistic = calculate_expected_ACE(resampled_data, repeat_counts)
    
    # Store the resampled statistic
    resampled_statistics[i] = resampled_statistic

# Calculate the lower and upper bounds of the confidence interval
alpha = 1 - confidence_level
lower_bound = np.percentile(resampled_statistics, 100 * alpha / 2)
upper_bound = np.percentile(resampled_statistics, 100 * (1 - alpha / 2))

# Print the results
print(f"(bootstrapped) CI using expected value ({confidence_level*100:2.1f}%): [{lower_bound:3.2f}, {upper_bound:3.2f}]")




# Specify the confidence level (e.g., 90% CI)
confidence_level = 0.54

print("")
print(f"Statistics for {sorted_years[0]} - {sorted_years[-1]} excluding years with season ACE < realtime ACE:")
print(f"μ: {ace_mean:3.2f}")

# not as useful since it doesn't handle constraints well so don't use general statistics below for CI:
#print("σ:", ace_stddev)
#print("Z score for expected ACE:", z_score)
#ci_lower = expected_ACE - (critical_value * ace_stddev)
#ci_upper = expected_ACE + (critical_value * ace_stddev)
#print(f"CI({confidence_level}) = [ {ci_lower} , {ci_upper} ]")


# use bootstrapping instead for CI

# Number of bootstrap samples to generate
num_samples = 10000

# Calculate the statistic of interest (e.g., mean) for the original data
original_statistic = ace_mean

# Initialize an array to store resampled statistics
resampled_statistics = np.empty(num_samples)

# Perform bootstrapping
for i in range(num_samples):
    # Generate a resampled dataset by sampling with replacement
    resampled_data = np.random.choice(pop_ace_values, size=len(pop_ace_values), replace=True)
    
    # Calculate the statistic of interest for the resampled dataset
    resampled_statistic = np.mean(resampled_data)
    
    # Store the resampled statistic
    resampled_statistics[i] = resampled_statistic

# Calculate the lower and upper bounds of the confidence interval
alpha = 1 - confidence_level
lower_bound = np.percentile(resampled_statistics, 100 * alpha / 2)
upper_bound = np.percentile(resampled_statistics, 100 * (1 - alpha / 2))

# Print the results
print(f"(bootstrapped) CI using mean ({confidence_level*100:2.1f}%): [{lower_bound:3.2f}, {upper_bound:3.2f}]")


# expect the tuned decay rate should automatically increase as the season passes
#   this means, later in the season it will be more selective in using years that have ACE that are very close to the present
#   as opposed to early in the season when the years will be more of an average
# for instance on Sept 24 06Z it was 0.147, and on Sept 25 00Z it was 0.171
print("")

# diagnostics for model predictiveness as the season progresses
expected_ACE_upper = expected_ACE + rmse_final_tuning
expected_ACE_lower = expected_ACE - rmse_final_tuning
#print(f"Range using RMSE (without validation RMSE): {expected_ACE_lower} : {expected_ACE_upper}")
expected_ACE_upper_with_validation = expected_ACE + rmse_final_tuning + diff_rmse
expected_ACE_lower_with_validation = expected_ACE - rmse_final_tuning - diff_rmse
#print(f"Range using RMSE (with validation RMSE):    {expected_ACE_lower_with_validation} : {expected_ACE_upper_with_validation}")


Forecasts until: 2023-09-25 00:00:00
ACE, Real-time + storm model forecast: 120.44
Tuned decay rate: 0.171

Expected ACE (weight historical ACE using exponentially decay): 168.00
(bootstrapped) CI using expected value (70.0%): [160.48, 237.50]

Statistics for 1851 - 2022 excluding years with season ACE < realtime ACE:
μ: 164.27
(bootstrapped) CI using mean (54.0%): [160.02, 168.31]

